In [1]:
import pandas as pd
import mysql.connector
import getpass,os

from mysql_connect import MySqlConnection

import numpy as np
import pandas as pd
import os
from pathlib import Path


In [2]:

movie_df = pd.read_csv('movies.csv', encoding='ISO-8859-1', on_bad_lines='skip', sep=';')
movie_df.dropna(inplace=True)
movie_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3795 entries, 0 to 3999
Data columns (total 15 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   name      3795 non-null   object 
 1   rating    3795 non-null   object 
 2   genre     3795 non-null   object 
 3   year      3795 non-null   int64  
 4   released  3795 non-null   object 
 5   score     3795 non-null   float64
 6   votes     3795 non-null   float64
 7   director  3795 non-null   object 
 8   writer    3795 non-null   object 
 9   star      3795 non-null   object 
 10  country   3795 non-null   object 
 11  budget    3795 non-null   int64  
 12  gross     3795 non-null   float64
 13  company   3795 non-null   object 
 14  runtime   3795 non-null   float64
dtypes: float64(4), int64(2), object(9)
memory usage: 474.4+ KB


In [3]:
### Mysql Server Connection 

budget_schema_config = {
    "host":"127.0.0.1",
    "user":"analysis"

}

mysql_conn = MySqlConnection(**budget_schema_config)
mysql_conn.connect()

MySql Server Connection Established...

Connection Details:
User --> analysis@localhost
Date Time --> 2025-01-10 12:00:07 PM


### Finding nth_value

In [6]:
## Connecting to Campusx Database
mysql_conn.select_database(db_name='Campusx')

Connected to database: Campusx


In [9]:
mysql_conn.select_table_query('select * from marks')

,student_id,name,branch,marks
0,1,Nitish,EEE,82
1,2,Rishabh,EEE,91
2,3,Anukant,EEE,69
3,4,Rupesh,EEE,55
4,5,Shubham,CSE,78
5,6,Ved,CSE,43
6,7,Deepak,CSE,98
7,8,Arpan,CSE,95
8,9,Vinay,ECE,95
9,10,Ankit,ECE,88


In [20]:
sql = """
select * 
from marks
order by marks desc;
""" 
mysql_conn.select_table_query(sql)

,student_id,name,branch,marks
0,7,Deepak,CSE,98
1,8,Arpan,CSE,95
2,9,Vinay,ECE,95
3,12,Rohit,ECE,95
4,2,Rishabh,EEE,91
5,10,Ankit,ECE,88
6,1,Nitish,EEE,82
7,11,Anand,ECE,81
8,5,Shubham,CSE,78
9,13,Prashant,MECH,75


In [17]:
sql = """
select *,
first_value(marks) over(order by marks desc)
from marks limit 1;
""" 
mysql_conn.select_table_query(sql)

,student_id,name,branch,marks,first_value(marks) over(order by marks desc)
0,7,Deepak,CSE,98,98


In [19]:
sql = """
select *,
last_value(marks) over(order by marks)
from marks limit 1;
""" 
mysql_conn.select_table_query(sql)

,student_id,name,branch,marks,last_value(marks) over(order by marks)
0,15,Sunny,MECH,39,39


In [54]:
## Min Max Avg Sum 
sql = """ 
select *,
min(marks) over(partition by branch) as Min,
max(marks) over(partition by branch) as Max,
avg(marks) over(partition by branch) as Avg,
sum(marks) over(partition by branch) as Sum
from marks
order by student_id
"""
mysql_conn.select_table_query(sql)

,student_id,name,branch,marks,Min,Max,Avg,Sum
0,1,Nitish,EEE,82,55,91,74.2500,297
1,2,Rishabh,EEE,91,55,91,74.2500,297
2,3,Anukant,EEE,69,55,91,74.2500,297
3,4,Rupesh,EEE,55,55,91,74.2500,297
4,5,Shubham,CSE,78,43,98,78.5000,314
5,6,Ved,CSE,43,43,98,78.5000,314
6,7,Deepak,CSE,98,43,98,78.5000,314
7,8,Arpan,CSE,95,43,98,78.5000,314
8,9,Vinay,ECE,95,81,95,89.7500,359
9,10,Ankit,ECE,88,81,95,89.7500,359


In [55]:
## Rank 
sql = """ 
select *,
rank() over(partition by branch order by marks desc) 
from marks
"""
mysql_conn.select_table_query(sql)

,student_id,name,branch,marks,rank() over(partition by branch order by marks desc)
0,7,Deepak,CSE,98,1
1,8,Arpan,CSE,95,2
2,5,Shubham,CSE,78,3
3,6,Ved,CSE,43,4
4,9,Vinay,ECE,95,1
5,12,Rohit,ECE,95,1
6,10,Ankit,ECE,88,3
7,11,Anand,ECE,81,4
8,2,Rishabh,EEE,91,1
9,1,Nitish,EEE,82,2


In [56]:
## Dense Rank 
sql = """ 
select *,
dense_rank() over(partition by branch order by marks desc) 
from marks
"""
mysql_conn.select_table_query(sql)

,student_id,name,branch,marks,dense_rank() over(partition by branch order by marks desc)
0,7,Deepak,CSE,98,1
1,8,Arpan,CSE,95,2
2,5,Shubham,CSE,78,3
3,6,Ved,CSE,43,4
4,9,Vinay,ECE,95,1
5,12,Rohit,ECE,95,1
6,10,Ankit,ECE,88,2
7,11,Anand,ECE,81,3
8,2,Rishabh,EEE,91,1
9,1,Nitish,EEE,82,2


In [57]:
## Row Number
sql = """ 
select *,
row_number () over() as rnk
from marks
"""
mysql_conn.select_table_query(sql)

,student_id,name,branch,marks,rnk
0,1,Nitish,EEE,82,1
1,2,Rishabh,EEE,91,2
2,3,Anukant,EEE,69,3
3,4,Rupesh,EEE,55,4
4,5,Shubham,CSE,78,5
5,6,Ved,CSE,43,6
6,7,Deepak,CSE,98,7
7,8,Arpan,CSE,95,8
8,9,Vinay,ECE,95,9
9,10,Ankit,ECE,88,10


In [58]:
## Row Number partition by branch
sql = """ 
select *,
row_number() over(partition by branch) row_num
from marks;

"""
mysql_conn.select_table_query(sql)

,student_id,name,branch,marks,row_num
0,5,Shubham,CSE,78,1
1,6,Ved,CSE,43,2
2,7,Deepak,CSE,98,3
3,8,Arpan,CSE,95,4
4,9,Vinay,ECE,95,1
5,10,Ankit,ECE,88,2
6,11,Anand,ECE,81,3
7,12,Rohit,ECE,95,4
8,1,Nitish,EEE,82,1
9,2,Rishabh,EEE,91,2


### Quering Zomato Table

In [59]:
mysql_conn.select_database(db_name='Zomato')

Connected to database: Zomato


In [63]:
## Each Customer Each Month Expense
sql = """ 
select *, month(date), monthname(date) from orders;
"""
mysql_conn.select_table_query(sql)

,index,order_id,user_id,r_id,amount,date,partner_id,delivery_time,delivery_rating,restaurant_rating,month(date),monthname(date)
0,0,1001,1,1,550,2022-05-10,1,25,5,3.0,5,May
1,1,1002,1,2,415,2022-05-26,1,19,5,2.0,5,May
2,2,1003,1,3,240,2022-06-15,5,29,4,NaN,6,June
3,3,1004,1,3,240,2022-06-29,4,42,3,5.0,6,June
4,4,1005,1,3,220,2022-07-10,1,58,1,4.0,7,July
5,5,1006,2,1,950,2022-06-10,2,16,5,NaN,6,June
6,6,1007,2,2,530,2022-06-23,3,60,1,5.0,6,June
7,7,1008,2,3,240,2022-07-07,5,33,4,5.0,7,July
8,8,1009,2,4,300,2022-07-17,4,41,1,NaN,7,July
9,9,1010,2,5,650,2022-07-31,1,67,1,4.0,7,July


In [80]:
## Each Customer Each Month Expense
sql = """ 
select * from (
                select monthname(date) month , user_id, sum(amount) total ,
                rank() over (partition by monthname(date) order by sum(amount) desc) month_rank
                from orders
                group by 1,2    
               ) t
where t.month_rank <3
order by t.month desc
"""
mysql_conn.select_table_query(sql)

,month,user_id,total,month_rank
0,May,1,965,1
1,May,3,860,2
2,June,2,1480,1
3,June,4,800,2
4,July,5,3035,1
5,July,2,1190,2


### Finding nth_value

In [21]:
mysql_conn.connection_close()

MySql Connection Closed.........2025-01-10 01:24:06 PM

